Create a Kafka producer client using the given config, and produce 100 messages using `faker` of 
```
{
    'id': uuid4(),
    'name': name(),
    'email': email(),
    'timestamp': time.time()
}
```
to topic called `msds682.day3`

In [1]:
import json
import os
import time

from confluent_kafka import Producer
from dotenv import load_dotenv
from faker import Faker

In [2]:
load_dotenv()

True

In [3]:
conf = {
        'bootstrap.servers': os.environ['CONFLUENT_SERVER'],
        'security.protocol': 'SASL_SSL',
        'sasl.mechanism': 'PLAIN',
        'sasl.username': os.environ['CONFLUENT_API_KEY'],
        'sasl.password': os.environ['CONFLUENT_API_SECRET'],
    }

In [4]:
# Create the producer instance
producer = Producer(conf)

# Initialize data generator
fake = Faker()

In [5]:
def delivery_report(err, msg):
    """
    Called once for each message produced to indicate the delivery status.
    """
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message {msg.key()} delivered to topic '{msg.topic()}' to Partition {msg.partition()}")

In [6]:
for i in range(10):
    # Generate mock user data
    user_data = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'timestamp': time.time()
    }
    
    # Serialize the data to a JSON string
    json_data = json.dumps(user_data)
    
    # Produce the message to the Kafka topic
    # `on_delivery` specifies the callback function.
    producer.produce(
        topic='msds682.day3',
        key=str(user_data['id']),
        value=json_data,
        on_delivery=delivery_report
    )
    
    # Flush any outstanding or buffered messages to the Kafka broker.
    producer.poll(0)
    
    # Pause for a moment to simulate a real-time stream
    time.sleep(1)

producer.flush()

%6|1761151682.619|GETSUBSCRIPTIONS|rdkafka#producer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to 1UxjwbflTNGJgHal3KuUKQ


Message b'15caa426-e9e4-4289-a5ba-4cd55d196051' delivered to topic 'msds682.day3' to Partition 2
Message b'13dd40a3-e5c0-4f86-b061-cd23415d1e9d' delivered to topic 'msds682.day3' to Partition 2
Message b'1cbce282-7502-4c5a-97a6-3256fe7401e0' delivered to topic 'msds682.day3' to Partition 2
Message b'eb0d9348-edf9-4db6-9638-df7379f4a79e' delivered to topic 'msds682.day3' to Partition 0
Message b'5a64200a-5bc9-49ee-acbd-02f4365a7d29' delivered to topic 'msds682.day3' to Partition 4
Message b'b99794fa-f75c-4722-8d53-5d31fb7c7d4d' delivered to topic 'msds682.day3' to Partition 0
Message b'89663761-2c1c-47b5-b916-9186970dffea' delivered to topic 'msds682.day3' to Partition 0
Message b'70431632-aa2c-4094-b798-a6cb984ae027' delivered to topic 'msds682.day3' to Partition 0
Message b'eaf45680-b3da-4dd5-bd47-ab593d969ace' delivered to topic 'msds682.day3' to Partition 3
Message b'7442f399-1683-4ef9-bca8-fb6b5c2dd1d7' delivered to topic 'msds682.day3' to Partition 3


0

Example - What happens without poll() and flush()?

In [9]:
while True:
    user_data = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'timestamp': time.time()
    }
    
    json_data = json.dumps(user_data)
    
    producer.produce(
        topic='msds682.day3',
        key=str(user_data['id']),
        value=json_data,
    )

BufferError: Local: Queue full